<a href="https://colab.research.google.com/github/yananma/5_programs_per_day/blob/master/0521.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!cat /usr/local/cuda/version.txt

CUDA Version 10.0.130


In [0]:
!pip install mxnet-cu100 d2lzh

In [3]:
from mxnet import nd 

X, W_xh = nd.random.normal(shape=(3, 1)), nd.random.normal(shape=(1, 4))
H, W_hh = nd.random.normal(shape=(3, 4)), nd.random.normal(shape=(4, 4))
nd.dot(X, W_xh) + nd.dot(H, W_hh)


[[ 5.0373516   2.6754622  -1.6607479  -0.40628886]
 [ 0.948454    0.46941757 -1.1866101  -1.180677  ]
 [-1.151402    0.83730245 -2.1974368  -5.248016  ]]
<NDArray 3x4 @cpu(0)>

In [4]:
nd.dot(nd.concat(X, H, dim=1), nd.concat(W_xh, W_hh, dim=0))


[[ 5.0373516   2.6754622  -1.6607479  -0.40628862]
 [ 0.94845396  0.46941754 -1.1866102  -1.1806769 ]
 [-1.151402    0.8373025  -2.1974368  -5.248016  ]]
<NDArray 3x4 @cpu(0)>

## 语言模型数据集 ( 歌词 )

In [7]:
from google.colab import files 
uploaded = files.upload()

Saving jaychou_lyrics.txt.zip to jaychou_lyrics.txt.zip


In [8]:
from mxnet import nd 
import random 
import zipfile 

with zipfile.ZipFile('jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')
corpus_chars[:40]

'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每'

In [0]:
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[0:10000]

In [10]:
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
vocab_size

1027

In [11]:
corpus_indices = [char_to_idx[char] for char in corpus_chars]
sample = corpus_indices[:20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [918, 734, 559, 558, 158, 760, 571, 918, 734, 656, 845, 223, 659, 894, 751, 95, 571, 918, 734, 656]


In [0]:
def data_iter_random(corpus_indices, batch_size, num_steps, ctx=None):
    num_examples = (len(corpus_indices) - 1) // num_steps 
    epoch_size = num_examples // batch_size 
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)

    def _data(pos):
        return corpus_indices[pos: pos + num_steps]
    
    for i in range(epoch_size):
        i = i * batch_size 
        batch_indices = example_indices[i: i + batch_size]
        X = [_data(j * num_steps) for j in batch_indices]
        Y = [_data(j * num_steps + 1) for j in batch_indices]
        yield nd.array(X, ctx), nd.array(Y, ctx)

In [13]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY', Y, '\n')

X:  
[[12. 13. 14. 15. 16. 17.]
 [18. 19. 20. 21. 22. 23.]]
<NDArray 2x6 @cpu(0)> 
Y 
[[13. 14. 15. 16. 17. 18.]
 [19. 20. 21. 22. 23. 24.]]
<NDArray 2x6 @cpu(0)> 

X:  
[[ 6.  7.  8.  9. 10. 11.]
 [ 0.  1.  2.  3.  4.  5.]]
<NDArray 2x6 @cpu(0)> 
Y 
[[ 7.  8.  9. 10. 11. 12.]
 [ 1.  2.  3.  4.  5.  6.]]
<NDArray 2x6 @cpu(0)> 



In [0]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx=None):
    corpus_indices = nd.array(corpus_indices, ctx=ctx)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size 
    indices = corpus_indices[0: batch_size*batch_len].reshape((batch_size, batch_len))
    epoch_size = (batch_len - 1) // num_steps
    for i in range(epoch_size):
        i = i * num_steps 
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [15]:
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY: ', Y, '\n')

X:  
[[ 0.  1.  2.  3.  4.  5.]
 [15. 16. 17. 18. 19. 20.]]
<NDArray 2x6 @cpu(0)> 
Y:  
[[ 1.  2.  3.  4.  5.  6.]
 [16. 17. 18. 19. 20. 21.]]
<NDArray 2x6 @cpu(0)> 

X:  
[[ 6.  7.  8.  9. 10. 11.]
 [21. 22. 23. 24. 25. 26.]]
<NDArray 2x6 @cpu(0)> 
Y:  
[[ 7.  8.  9. 10. 11. 12.]
 [22. 23. 24. 25. 26. 27.]]
<NDArray 2x6 @cpu(0)> 



In [16]:
!nvidia-smi

Sun Nov 10 05:47:54 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## 6.4 循环神经网络的从零开始实现

In [0]:
!mkdir ../data

In [22]:
!ls ..

bin	 data	  etc	lib32  mnt   root  srv	  tensorflow-2.0.0  usr
boot	 datalab  home	lib64  opt   run   swift  tmp		    var
content  dev	  lib	media  proc  sbin  sys	  tools


In [0]:
!cp jaychou_lyrics.txt.zip ../data/

In [26]:
!ls ../data/

jaychou_lyrics.txt.zip


In [0]:
import d2lzh as d2l 
import math 
from mxnet import autograd, nd 
from mxnet.gluon import loss as gloss 
import time 

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

In [30]:
nd.one_hot(nd.array([0, 2]), vocab_size)


[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
<NDArray 2x1027 @cpu(0)>

In [32]:
def to_onehot(X, size):
    return [nd.one_hot(x, size) for x in X.T]

X = nd.arange(10).reshape((2, 5))
inputs = to_onehot(X, vocab_size)
len(inputs), inputs[0].shape

(5, (2, 1027))

In [37]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size 
ctx = d2l.try_gpu()
print('will use ', ctx)

def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)

    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = nd.zeros(num_hiddens, ctx=ctx)
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)

    params = [W_xh, W_hh, b_h, W_hq, b_q] 
    for param in params:
        param.attach_grad()
    return params 

will use  gpu(0)


In [0]:
def init_rnn_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), )

In [0]:
def rnn(inputs, state, params):
    W_xh, W_hh, b_h, W_hq, b_q = params 
    H, = state 
    outputs = []
    for X in inputs:
        H = nd.tanh(nd.dot(X, W_xh) + nd.dot(H, W_hh) + b_h)
        Y = nd.dot(H, W_hq) + b_q 
        outputs.append(Y)
    return outputs, (H, )

In [40]:
state = init_rnn_state(X.shape[0], num_hiddens, ctx)
inputs = to_onehot(X.as_in_context(ctx), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
len(outputs), outputs[0].shape, state_new[0].shape 

(5, (2, 1027), (2, 256))

In [0]:
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state, num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, ctx)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        X = to_onehot(nd.array([output[-1]], ctx=ctx), vocab_size)
        (Y, state) = rnn(X, state, params)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])

In [44]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx)

'分开等盘喝换距驳蕃车响倒'

In [0]:
def grad_clipping(params, theta, ctx):
    norm = nd.array([0], ctx)
    for param in params:
        norm += (param.grad ** 2).sum()
    norm = norm.sqrt().asscalar()
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm 

In [0]:
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens, 
             vocab_size, ctx, corpus_indices, idx_to_char, 
             char_to_idx, is_random_iter, num_epochs, num_steps, 
             lr, clipping_theta, batch_size, pred_period, 
             pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random 
    else:
        data_iter_fn = d2l.data_iter_consecutive 
    params = get_params()
    loss = gloss.SoftmaxCrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:
            state = init_rnn_state(batch_size, num_hiddens, ctx)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, ctx)
        for X, Y in data_iter:
            if is_random_iter:
                state = init_rnn_state(batch_size, num_hiddens, ctx)
            else:
                for s in state:
                    s.detach()
            with autograd.record():
                inputs = to_onehot(X, vocab_size)
                (outputs, state) = rnn(inputs, state, params)
                outputs = nd.concat(*outputs, dim=0)
                y = Y.T.reshape((-1, ))
                l = loss(outputs, y).mean()
            l.backward()
            grad_clipping(params, clipping_theta, ctx)
            d2l.sgd(params, lr, 1)
            l_sum += l.asscalar() * y.size 
            n += y.size 

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start
            ))
            for prefix in prefixes:
                print(' -', predict_rnn(
                    prefix, pred_len, rnn, params, init_rnn_state, 
                    num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx
                ))

In [0]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2 
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [55]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens, vocab_size, ctx, corpus_indices, 
        idx_to_char, char_to_idx, True, num_epochs, num_steps, lr, clipping_theta, batch_size, 
        pred_period, pred_len, prefixes)

epoch 50, perplexity 71.381833, time 0.43 sec
 - 分开 我不要 爱你我 说子我 别子我 别子我 别子我 别子我 别子我 别子我 别子我 别子我 别子我 别
 - 不分开 我有的 爱你我 我不要 一颗我 别子我 别子我 别子我 别子我 别子我 别子我 别子我 别子我 别
epoch 100, perplexity 9.813439, time 0.43 sec
 - 分开 一颗两步三 用在它在留 仙人在停落 这人在停落 这人在停落 这人在停落 这人在停落 这人在停落 这
 - 不分开久 我不能再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我该了这节奏 不知后觉 
epoch 150, perplexity 2.938046, time 0.43 sec
 - 分开 爱家用双截棍 哼哼哈兮 习使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快
 - 不分开扫 然后你看去 慢慢温开天 仙人在怕羞 蜥蝪横著走 这样什么奇 蜥蝪横著走 蜥不懂 走 懂里什么奇 
epoch 200, perplexity 1.599412, time 0.43 sec
 - 分开 一步两双截 辛辛它在抽屉 它所拥有的只剩下回忆 相爱还说前离字日的平墙 夕著斜木映的屋内还弥漫 姥
 - 不分开扫 然后将过去 慢慢在习前 你在在元前 蜥蝪 什么却 旧词依间截 几辛盘苦 全家怕日出 白色蜡烛 温
epoch 250, perplexity 1.299030, time 0.43 sec
 - 分开 一步两停哭 还钩它满走 这样就反驳 到底拽什么 懂不懂篮球 这种不要走 三对三斗牛 三分球 它在空
 - 不分开期 然后将过去 慢慢温满天 一朵一朵因 三被在著妥 这里就反说 懂里什么奇 三对就什么 懂不懂篮球 


In [56]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens, vocab_size, ctx, corpus_indices, 
        idx_to_char, char_to_idx, False, num_epochs, num_steps, lr, clipping_theta, batch_size, 
        pred_period, pred_len, prefixes)

epoch 50, perplexity 62.303389, time 0.43 sec
 - 分开 我想要这 你使了双 我有一场 如果我 别怪我 我的我有 你的让空 我有了空 我有了外 在谁了外 泪
 - 不分开 我不能再 你有我有 你谁了外 在谁的可 感狂的可 感狂的可 感狂的可 感狂的可 感狂的可 感狂的可
epoch 100, perplexity 7.131350, time 0.42 sec
 - 分开 你想了 是你我抬起头 有话去对医药箱说 别怪我 别怪我 说你怎么面对我 甩开球我满腔的怒火 我想揍
 - 不分开柳 你想经这开我 不知不觉 我已了这节奏 我该好好生活 不知不觉 我已经这节奏 我该好这生活 不知我
epoch 150, perplexity 2.039645, time 0.42 sec
 - 分开 这说我 谁是我 印地安的传说 还真是 瞎透了 什么都有 沙漠之 怎么她著停留 一直在停留 谁让它停
 - 不分开柳 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后兮 快使用双截棍 哼哼哈兮 
epoch 200, perplexity 1.271316, time 0.43 sec
 - 分开 问候堂 是属于那年代白墙黑瓦的淡淡的忧伤 消失的 旧时光 一九四三 回头看 的片段 有一些风霜 老
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 后知后觉 
epoch 250, perplexity 1.168021, time 0.42 sec
 - 分开 问候堂依旧在的事我 泪不休 语沉默 娘子她人在江南等我 泪不休 语沉默娘子 娘子却依旧每日 折一枝
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生


## 6.5 循环神经网络的简洁实现

In [0]:
import d2lzh as d2l 
import math 
from mxnet import autograd, gluon, init, nd 
from mxnet.gluon import loss as Gloss, nn, rnn 
import time 

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

In [0]:
num_hiddens = 256 
rnn_layer = rnn.RNN(num_hiddens)
rnn_layer.initialize()

In [59]:
batch_size = 2 
state = rnn_layer.begin_state(batch_size=batch_size)
state[0].shape 

(1, 2, 256)

In [60]:
num_steps = 35 
X = nd.random.uniform(shape=(num_steps, batch_size, vocab_size))
Y, state_new = rnn_layer(X, state)
Y.shape, len(state_new), state_new[0].shape 

((35, 2, 256), 1, (1, 2, 256))

In [0]:
class RNNModel(nn.Block):
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer 
        self.vocab_size = vocab_size 
        self.dense = nn.Dense(vocab_size)

    def forward(self, inputs, state):
        X = nd.one_hot(inputs.T, self.vocab_size)
        Y, state = self.rnn(X, state)
        output = self.dense(Y.reshape((-1, Y.shape[-1])))
        return output, state 
    
    def begin_state(self, *args, **kwargs):
        return self.rnn.begin_state(*args, **kwargs)

In [0]:
def predict_rnn_gluon(prefix, num_chars, model, vocab_size, ctx, idx_to_char, char_to_idx):
    state = model.begin_state(batch_size=1, ctx=ctx)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        X = nd.array([output[-1]], ctx=ctx).reshape((1, 1))
        (Y, state) = model(X, state)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y.argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])

In [65]:
ctx = d2l.try_gpu()
model = RNNModel(rnn_layer, vocab_size)
model.initialize(force_reinit=True, ctx=ctx)
predict_rnn_gluon('分开', 10, model, vocab_size, ctx, idx_to_char, char_to_idx)

'分开帅物细疼物细疼物细疼'

In [0]:
def train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx, corpus_indices, idx_to_char, char_to_idx, 
            num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, prefixes):
    loss = gloss.SoftmaxCrossEntropyLoss()
    model.initialize(ctx=ctx, force_reinit=True, init=init.Normal(0.01))
    trainer = gluon.Trainer(model.collect_params(), 'sgd', {'learning_rate': lr, 'momentum': 0, 'wd': 0})

    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = d2l.data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx)
        state = model.begin_state(batch_size=batch_size, ctx=ctx)
        for X, Y in data_iter:
            for s in state:
                s.detach()
            with autograd.record():
                (output, state) = model(X, state)
                y = Y.T.reshape((-1, ))
                l = loss(output, y).mean()
            l.backward()
            params = [p.data() for p in model.collect_params().values()]
            d2l.grad_clipping(params, clipping_theta, ctx)
            trainer.step(1)
            l_sum += l.asscalar() * y.size 
            n += y.size 

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' %(
                epoch + 1, math.exp(l_sum / n), time.time() - start
            ))
            for prefix in prefixes:
                print(' _', predict_rnn_gluon(
                    prefix, pred_len, model, vocab_size, ctx, idx_to_char, char_to_idx
                ))

In [68]:
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e2, 1e-2 
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx, corpus_indices, idx_to_char, char_to_idx, 
            num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 77.114559, time 0.12 sec
 _ 分开 我不能再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
 _ 不分开 我想我这想你的怒场就 印地在有多 我不能再想 我不要再想 我不要再想 我不要再想 我不要再想 我不
epoch 100, perplexity 12.234623, time 0.11 sec
 _ 分开 娘子我 你你我 每你怎么这对我 甩开球我满腔的怒火 我想揍你已经很久银像想我 别不我 说你想我这辈
 _ 不分开 我一著这样牵着你起想想我爱见你是 快不 你想 我爱要再想 我不要再想 我不能再想 我不能再想 我不
epoch 150, perplexity 3.961371, time 0.12 sec
 _ 分开 平候堂 是属于明年 白有的在丽 你的完美主义 太彻底 分我连恨都难以 我 想和你在语日 我只想你的
 _ 不分开 我一你的叹息 对作依依不舍 连隔壁 干什么 我被天这样 老唱盘 旧皮箱 装属了明信 白在的有丽 我
epoch 200, perplexity 2.257617, time 0.11 sec
 _ 分开 娘养的黑猫笑起来像哭 啦啦啦呜 用水晶球 你底子空 恨人己痛 在你跟痛 你不没 连一句珍 停止转 
 _ 不分开觉 你在我面辈泪 不要再这样打我妈妈 我的伤口被你看封 誓言太沉重泪被纵知 脸上再涌宙 有一条热昏头
epoch 250, perplexity 1.779178, time 0.12 sec
 _ 分开 什么兵 是属于那年代 所有人看着我 抛物线进球 单手过人运球 篮下妙传出手 漂亮 失去意义 戒指在
 _ 不分开  你知世这坦堡 泛只林跟 悲你马 隐前 篮下的客栈人多 牧草有没有 我马儿有些瘦 天涯尽头 满脸风


## 6.7 门控循环单元 ( GRU )

In [0]:
import d2lzh as d2l 
from mxnet import nd 
from mxnet.gluon import rnn 

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

In [0]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size 
ctx = d2l.try_gpu()

def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)

    def _three():
        return (_one((num_inputs, num_hiddens)), 
            _one((num_hiddens, num_hiddens)), 
            nd.zeros(num_hiddens, ctx=ctx))
    
    W_xz, W_hz, b_z = _three()
    W_xr, W_hr, b_r = _three()
    W_xh, W_hh, b_h = _three()
    
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    
    params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params 

In [0]:
def init_gru_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), )

In [0]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params 
    H, = state 
    outputs = []
    for X in inputs:
        Z = nd.sigmoid(nd.dot(X, W_xz) + nd.dot(H, W_hz) + b_z)
        R = nd.sigmoid(nd.dot(X, W_xr) + nd.dot(H, W_hr) + b_r)
        H_tilda = nd.tanh(nd.dot(X, W_xh) + nd.dot(R * H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda 
        Y = nd.dot(H, W_hq) + b_q 
        outputs.append(Y)
    return outputs, (H,)

In [100]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2 
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']
d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens, vocab_size, ctx, corpus_indices, idx_to_char, char_to_idx, False,
            num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 153.516774, time 0.99 sec
 - 分开 我想你的让我 爱爱人 我不的让我 爱爱人 我不的让我 爱爱人 我不的让我 爱爱人 我不的让我 爱爱
 - 不分开 我想你的让我 爱爱人 我不的让我 爱爱人 我不的让我 爱爱人 我不的让我 爱爱人 我不的让我 爱爱
epoch 80, perplexity 32.849917, time 0.97 sec
 - 分开 我想要这样 我有就这样 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
 - 不分开 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
epoch 120, perplexity 5.792713, time 0.99 sec
 - 分开 我想要你的微笑有天都能风看    在我胸口泪著 像这样的让快果我 能地的让我疯狂的可爱女人 坏坏的
 - 不分开 我知不觉 我跟了这节奏 后知后觉 我该好好生活 我该好好生活 不知不觉 你已经离开我 不知不觉 我
epoch 160, perplexity 1.768622, time 0.99 sec
 - 分开 我想要的没有笑每天乡看到到 就是在最不能 到你到回面对我 甩开 我想就这了吧着你 不想太多 你来一
 - 不分开 不知再觉 你来一句奏默 后知后觉 我该好好生活 我该好好生活 静静悄悄默默离开 陷入了危险边缘Ba


In [103]:
gru_layer = rnn.GRU(num_hiddens)
model = d2l.RNNModel(gru_layer, vocab_size)
train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx, corpus_indices, idx_to_char, char_to_idx, 
            num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 153.891103, time 0.12 sec
 _ 分开 我想你你的让我想想想想你你你的可爱女 我想你的让我想想想想你想你想你想你想你想你想你想你想你想你想
 _ 不分开 我想你你的让我想想想想你你你的可爱女 我想你的让我想想想想你想你想你想你想你想你想你想你想你想你想
epoch 80, perplexity 31.606348, time 0.12 sec
 _ 分开 我想要这样 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
 _ 不分开  没有你在我有 别发你的生快 我想要你 我不要 爱你的手不听 想要你的爱写 一定后 是你的手不多 
epoch 120, perplexity 4.693406, time 0.12 sec
 _ 分开 一直我 说你怎么面对我 别发球我满腔的怒火 我想揍你已经很久 别想躲 说你眼睛看着我 别发抖 快给
 _ 不分开  我知这这里着我 甩开球我满腔的怒火 我想揍你已经很久 别想躲 说你眼睛看着我 别发抖 快给我抬起
epoch 160, perplexity 1.451727, time 0.12 sec
 _ 分开 别弄 是你开的玩笑 想通 却又再考倒我 说散 你想很久了吧? 败给你的黑色幽默 说散 你想很久了吧
 _ 不分开  我来这样了我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生


## 6.8 长短期记忆 ( LSTM )

In [0]:
import d2lzh as d2l 
from mxnet import nd 
from mxnet.gluon import rnn  

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

In [0]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size 
ctx = d2l.try_gpu()

def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)

    def _three():
        return (_one((num_inputs, num_hiddens)), 
            _one((num_hiddens, num_hiddens)), 
            nd.zeros(num_hiddens, ctx=ctx))
    
    W_xi, W_hi, b_i = _three()
    W_xf, W_hf, b_f = _three()
    W_xo, W_ho, b_o = _three()
    W_xc, W_hc, b_c = _three()

    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    params = [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params 

In [0]:
def init_lstm_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), 
        nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx))

In [0]:
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params 
    (H, C) = state 
    outputs = []
    for X in inputs:
        I = nd.sigmoid(nd.dot(X, W_xi) + nd.dot(H, W_hi) + b_i)
        F = nd.sigmoid(nd.dot(X, W_xf) + nd.dot(H, W_hf) + b_f)
        O = nd.sigmoid(nd.dot(X, W_xo) + nd.dot(H, W_ho) + b_o)
        C_tilda = nd.tanh(nd.dot(X, W_xc) + nd.dot(H, W_hc) + b_c)
        C = F * C + I * C_tilda 
        H = O * C.tanh()
        Y = nd.dot(H, W_hq) + b_q 
        outputs.append(Y)
    return outputs, (H, C)

In [0]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2 
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [110]:
d2l.train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens, vocab_size, ctx, corpus_indices, idx_to_char,
            char_to_idx, False, num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, 
            pred_len, prefixes)

epoch 40, perplexity 211.511178, time 1.18 sec
 - 分开 我不的 我不 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不
 - 不分开 我不的我 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 
epoch 80, perplexity 67.897039, time 1.17 sec
 - 分开 我想你这你你 我想想你想我想 你不你 我不了 我不要 我不要 我不了 我不要 我不了 我不了 我不
 - 不分开 我想你这你你 我想要你想我 你不你 我不了 我不要 我不要 我不了 我不要 我不了 我不了 我不了
epoch 120, perplexity 15.597091, time 1.15 sec
 - 分开 我想你这生活 我知好这生活 后知后觉 我跟了这节奏 后知后觉 我该好好节活 我知好好生活 后知后觉
 - 不分开 我想要这样 我不要这样 我不好这生活 后知后觉 你跟了一个我 后知不觉 我该了好节活 我知好好生活
epoch 160, perplexity 4.250535, time 1.19 sec
 - 分开 我已带你 你跟的外人 后知后觉 我该好好节奏 我该好好生活 静静悄悄默默离开 陷入了危险边缘 我 
 - 不分开 我已经这生我 我知能 爱你 我想开难熬活 我该好这生活 静知悄觉默离离开 陷入了危险边缘B 心说我


In [112]:
lstm_layer = rnn.LSTM(num_hiddens)
model = d2l.RNNModel(lstm_layer, vocab_size)
d2l.train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx, corpus_indices, idx_to_char, char_to_idx, 
                num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, 
                prefixes)

epoch 40, perplexity 220.343904, time 0.14 sec
 - 分开 我不的 我 我不 我 我不 我 我不 我 我不 我 我不 我 我不 我 我不 我 我不 我 我不 
 - 不分开 我不的 我 我不 我 我不 我 我不 我 我不 我 我不 我 我不 我 我不 我 我不 我 我不 
epoch 80, perplexity 66.893844, time 0.14 sec
 - 分开 我想你你的你我 想想你 你你我 我不要这不我 你不不觉 我不要这不 我不要这 我不不好 我不不好 
 - 不分开 我想要你的你 我不要这 我不要这不 我不要这不 我不要这不 我不要这不 我不要这不 我不不好 我不
epoch 120, perplexity 13.965304, time 0.14 sec
 - 分开 我想你的太笑 一你 在小我 一九的碗  有一碗 旧片段 有一风 装有的风  有有苦 旧一了 装一风
 - 不分开我 你要你的微我不要 你说你的风 我一定到天 我不要我我 我要 我不  爱有人的我 你要就我 你你的
epoch 160, perplexity 3.680804, time 0.14 sec
 - 分开 说子 是是我的玩  有在起 是谁了在在手  有在起 是谁是双截极 所色在兮 快使用双截棍 哼哼哈兮
 - 不分开想你 想样的没样怎著么 也是一了 你想我难多在我有错错错搞错 拜托 我想是你的脑袋有问题 随便说说 
